In [104]:
import numpy as np
import pyproj
import folium

FT2M = 0.3048

In [105]:
TFC_DENS = 2e-10 # Traffic Density in aircraft/m^3 to visualise
NUM_AC = 10**2  # Number of aircraft per 2D layer
NUM_LAYERS = 3 # Number of vertical layers
Z_SHIFT = 1500 # Number of metres to shift each z layer for decluttering
VERT_SEP = 1000 * FT2M # Vertical separation in metres for between each layer
PROJ_ORIGIN_4326 = (-1.45, 50.75) # The south west origin for the map

In [106]:
trans = pyproj.Transformer.from_crs('epsg:4326', 'epsg:3857', always_xy=True)
inv_trans = pyproj.Transformer.from_crs('epsg:3857', 'epsg:4326', always_xy=True)
proj_origin_3857 = np.array(trans.transform(*PROJ_ORIGIN_4326))

In [107]:
tfc_dens_2d = TFC_DENS * VERT_SEP
tfc_spacing_2d = np.sqrt(1 / tfc_dens_2d)

num_ac_1d = np.ceil(np.sqrt(NUM_AC))

In [108]:
import random
get_colors = lambda n: ["#%06x" % random.randint(0, 0xFFFFFF) for _ in range(n)]

In [109]:
m = folium.Map(PROJ_ORIGIN_4326[::-1])
layer_colours = get_colors(NUM_LAYERS)

for z in range(NUM_LAYERS):
    xs = proj_origin_3857[0] + (z * Z_SHIFT) + np.arange(0, num_ac_1d) * tfc_spacing_2d
    ys = proj_origin_3857[1] + (z * Z_SHIFT) + np.arange(0, num_ac_1d) * tfc_spacing_2d

    ps = inv_trans.transform(xs, ys)

    pxs = ps[0]
    pys = ps[1]

    xx, yy = np.meshgrid(xs, ys)
    for px in pxs:
        for py in pys:
            folium.CircleMarker((py, px), radius=2, color=layer_colours[z]).add_to(m)



In [110]:
from IPython.display import Markdown as md
md(f'This will display a map showing the traffic corresponding to that traffic density. Each colour is a different layer separated by {VERT_SEP/FT2M} ft')

This will display a map showing the traffic corresponding to that traffic density. Each colour is a different layer separated by 1000.0 ft

You must change then re run the notebook to display this for other combinations of variables.

In [111]:
m